In [1]:
import pandas as pd
import numpy as np
import time


In [2]:
# Read dataset
gaugeColumnList = {}
gaugeColumnList["Mean"] = ["Mean Radius","Mean Texture","Mean Perimeter","Mean Area","Mean Smoothness","Mean Compactness","Mean Concavity","Mean Concave points","Mean Symmetry","Mean Fractal dimension"]
gaugeColumnList["SE"] = ["Radius SE","Texture SE","Perimeter SE","Area SE","Smoothness SE","Compactness SE","Concavity SE","Concave points SE","Symmetry SE","Fractal dimension SE"]
gaugeColumnList["Worst"] = ["Worst Radius","Worst Texture","Worst Perimeter","Worst Area","Worst Smoothness","Worst Compactness","Worst Concavity","Worst Concave points","Worst Symmetry","Worst Fractal dimension"]
gaugeColumnList["Result"] = ["Diagnosis"]
columnNames = []
columnNames.extend(["ID number","Diagnosis"])
columnNames.extend(gaugeColumnList["Mean"])
columnNames.extend(gaugeColumnList["SE"])
columnNames.extend(gaugeColumnList["Worst"])
featureNames = []
featureNames.extend(gaugeColumnList["Mean"])
featureNames.extend(gaugeColumnList["SE"])
featureNames.extend(gaugeColumnList["Worst"])
dataset_training = pd.read_csv("../Data/data.csv", header=None, names=columnNames)
# print(dataset_training.T)

In [3]:
colorList = ["red", "yellow"]
colors = {}
colors["M"] = colorList[0]
colors["B"] = colorList[1]
dataset_training["Color"] = dataset_training.apply(lambda row: colors[row.Diagnosis], axis=1)
colors["M"] = 0
colors["B"] = 1
dataset_training["Y1"] = dataset_training.apply(lambda row: colors[row.Diagnosis], axis=1)
colors["M"] = 1
colors["B"] = 0
dataset_training["Y2"] = dataset_training.apply(lambda row: colors[row.Diagnosis], axis=1)




In [4]:
 def initialize_parameters_deep(layer_dims):
    t = 1000 * time.time() # current time in milliseconds
    np.random.seed(int(t) % 2**32)
#     np.random.seed(3)
    parameters = {}
    L = len(layer_dims) # number of layers in the network

    for l in range(1, L):
        parameters["W" + str(l)] = np.random.randn(layer_dims[l], layer_dims[l - 1])
        parameters["b" + str(l)] = np.zeros((layer_dims[l], layer_dims[l - 1]))
        parameters["b" + str(l)] = np.zeros((layer_dims[l], 1))
    return parameters


In [5]:
def linear_activation_forward(A_prev, W, b, activation):
    (Z, linear_cache) = linear_forward(A_prev, W, b)
    if activation == "sigmoid":
        (A, activation_cache) = sigmoid(Z)
    elif activation == "relu":
        (A, activation_cache) = relu(Z)
    cache = (linear_cache, activation_cache)
    return A, cache


In [6]:
def linear_forward(A, W, b):
    Z = W.dot(A) + b
    cache = (A, W, b)
    return Z, cache

In [7]:
def L_model_forward(X, parameters):
    caches = []
    A = X
    L = len(parameters) // 2                  # number of layers in the neural network
    for l in range(1, L):
        A_prev = A 
        activation = "relu"
#         activation = "sigmoid"
        (A, cache) = linear_activation_forward(A_prev, parameters["W" + str(l)], parameters["b" + str(l)], activation)
        caches.append(cache)
    activation = "sigmoid"
    (AL, cache) = linear_activation_forward(A, parameters["W" + str(L)], parameters["b" + str(L)], activation)
    caches.append(cache)
    return AL, caches


In [8]:
def L_layer_model(X, Y, layers_dims, learning_rate = 0.0075, num_iterations = 3000, print_cost=False):
    print("Learning rate: {}".format(learning_rate))
    np.random.seed(1)
    costs = []                         # keep track of cost
    parameters = initialize_parameters_deep(layers_dims)
    for i in range(0, num_iterations):
        (AL, caches) = L_model_forward(X, parameters)
        cost = compute_cost(AL, Y)
        grads = L_model_backward(AL, Y, caches)
        parameters = update_parameters(parameters, grads, learning_rate)
        if print_cost and i % 100 == 0 or i == num_iterations - 1:
            print("Cost after iteration {}: {}".format(i, np.squeeze(cost)))
        if i % 100 == 0 or i == num_iterations:
            costs.append(cost)
    return parameters, costs


In [9]:
def relu(Z):
  A = np.maximum(0.0, Z)
  return (A, Z)

In [10]:
def sigmoid(Z):
    EPSILON = 0.0000000000001
    A = 1 / (1 + np.exp(-Z) + EPSILON)
#     if (s == 1):
#         s -= EPSILON;
#     elif (s == 0):
#         s += EPSILON;
    return (A, Z)

In [11]:
def compute_cost(AL, Y):
    m = Y.shape[1]
    cost = -np.sum(Y * np.log(AL) + (1 - Y) * np.log(1 - AL), axis=1) / m
    return cost


In [12]:
def linear_activation_backward(dA, cache, activation):
    linear_cache, activation_cache = cache
    if activation == "relu":
        dZ = relu_backward(dA, activation_cache)
    elif activation == "sigmoid":
        dZ = sigmoid_backward(dA, activation_cache)
    (dA_prev, dW, db) = linear_backward(dZ, linear_cache)
    return dA_prev, dW, db


In [13]:
def L_model_backward(AL, Y, caches):
    grads = {}
    L = len(caches) # the number of layers
    m = AL.shape[1]
    Y = Y.reshape(AL.shape) # after this line, Y is the same shape as AL
    dAL = - (np.divide(Y, AL) - np.divide((1 - Y), (1 - AL)))
    current_cache = caches[L - 1]
    activation = "sigmoid"
    (dA_prev_temp, dW_temp, db_temp) = linear_activation_backward(dAL, current_cache, activation)
    grads["dA" + str(L-1)] = dA_prev_temp
    grads["dW" + str(L)] = dW_temp
    grads["db" + str(L)] = db_temp
    for l in reversed(range(L-1)):
        current_cache = caches[l]
        activation = "relu"
#         activation = "sigmoid"
        (dA_prev_temp, dW_temp, db_temp) = linear_activation_backward(grads["dA" + str(l + 1)], current_cache, activation)
        grads["dA" + str(l)] = dA_prev_temp
        grads["dW" + str(l + 1)] = dW_temp
        grads["db" + str(l + 1)] = db_temp
    return grads


In [14]:
def sigmoid_backward(dA, cache):
    Z = cache 
    s = 1/(1+np.exp(-Z))
    dZ = dA * s * (1-s)
    return dZ

In [15]:
def linear_backward(dZ, cache):
    A_prev, W, b = cache
    m = A_prev.shape[1]
    dW = dZ.dot(A_prev.T) / m
    db = np.sum(dZ, axis=1, keepdims=True) / m
    dA_prev = W.T.dot(dZ)
    return dA_prev, dW, db


In [16]:
def relu_backward(dA, cache):
    Z = cache
    dZ = np.array(dA, copy=True)
    dZ[Z <= 0] = 0
    return dZ

In [17]:
def update_parameters(params, grads, learning_rate):
    parameters = params.copy()
    L = len(parameters) // 2 # number of layers in the neural network
    for l in range(L):
        parameters["W" + str(l+1)] = params["W" + str(l + 1)] - learning_rate * grads["dW" + str(l + 1)]
        parameters["b" + str(l+1)] = params["b" + str(l + 1)] - learning_rate * grads["db" + str(l + 1)]
    return parameters


In [18]:
def normalizeData(data):
    return (data - np.min(data, axis=1, keepdims=True)) / (np.max(data, axis=1, keepdims=True) - np.min(data, axis=1, keepdims=True))


In [19]:
# n_x = len(gaugeColumnList["Mean"])
n_x = len(featureNames)
n_h = 3
n_y = 2
# layers_dims = (n_x, n_h, n_h, n_y)
layers_dims = (n_x, n_h, n_y)



In [20]:
train_x = normalizeData(dataset_training[gaugeColumnList["Mean"]].T.to_numpy())
train_x = normalizeData(dataset_training[featureNames].T.to_numpy())

# train_x = dataset_training[gaugeColumnList["Mean"]].T.to_numpy()
train_y = dataset_training[["Y1", "Y2"]].T.to_numpy()
learning_rate = 0.4
parameters, costs = L_layer_model(train_x, train_y, layers_dims, learning_rate, num_iterations = 2, print_cost = False)
print("Cost after first iteration: " + str(costs[0]))


Learning rate: 0.4
Cost after iteration 1: [0.78077816 0.64514921]
Cost after first iteration: [0.85549294 0.65223078]


In [21]:
learning_rate = 0.06
parameters, costs = L_layer_model(train_x, train_y, layers_dims, learning_rate, num_iterations = 1000, print_cost = True)


Learning rate: 0.06
Cost after iteration 0: [0.68582225 0.98660589]
Cost after iteration 100: [0.36404013 0.43867208]
Cost after iteration 200: [0.24036346 0.26236115]
Cost after iteration 300: [0.18830998 0.19731351]
Cost after iteration 400: [0.15978211 0.16457645]
Cost after iteration 500: [0.14092784 0.14389238]
Cost after iteration 600: [0.12733224 0.12932689]
Cost after iteration 700: [0.11707339 0.11848512]
Cost after iteration 800: [0.10912265 0.11015568]
Cost after iteration 900: [0.10281953 0.10359394]
Cost after iteration 999: [0.09774635 0.0983399 ]


In [22]:
(AL, caches) = L_model_forward(train_x, parameters)
cost = compute_cost(AL, train_y)
print(cost)
N = AL.shape[1]
argmax = np.argmax(AL, axis=0)
true_values = (train_y[1] == argmax).sum()
accuracy = true_values / N
print("{}/{}".format(true_values, N))
print("Accuracy: {}".format(accuracy))


[0.09769987 0.09829188]
554/569
Accuracy: 0.9736379613356766
